In [1]:
###6.4.1 多输入通道
import torch
from d2l import torch as d2l

def corr2d_multi_in(X, K):
    # 先遍历“X”和“K”的第0个维度（通道维度），再把它们加在一起
    return sum(d2l.corr2d(x, k) for x, k in zip(X, K))

In [2]:
X = torch.tensor([[[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]],
               [[1.0, 2.0, 3.0], [4.0, 5.0, 6.0], [7.0, 8.0, 9.0]]])
K = torch.tensor([[[0.0, 1.0], [2.0, 3.0]], [[1.0, 2.0], [3.0, 4.0]]])

corr2d_multi_in(X, K)

tensor([[ 56.,  72.],
        [104., 120.]])

In [3]:
###6.4.2. 多输出通道
def corr2d_multi_in_out(X, K):
    # 迭代“K”的第0个维度，每次都对输入“X”执行互相关运算。
    # 最后将所有结果都叠加在一起
    return torch.stack([corr2d_multi_in(X, k) for k in K], 0)

In [4]:
K = torch.stack((K, K + 1, K + 2), 0)
K.shape

torch.Size([3, 2, 2, 2])

In [5]:
corr2d_multi_in_out(X, K)

tensor([[[ 56.,  72.],
         [104., 120.]],

        [[ 76., 100.],
         [148., 172.]],

        [[ 96., 128.],
         [192., 224.]]])

In [7]:
###6.4.3 1x1卷积层
def corr2d_multi_in_out_1x1(X, K):
    c_i, h, w = X.shape
    c_o = K.shape[0]
    X = X.reshape((c_i, h * w))
    K = K.reshape((c_o, c_i))
    # 全连接层中的矩阵乘法
    Y = torch.matmul(K, X)
    return Y.reshape((c_o, h, w))

In [8]:
X = torch.normal(0, 1, (3, 3, 3))
K = torch.normal(0, 1, (2, 3, 1, 1))

Y1 = corr2d_multi_in_out_1x1(X, K)
Y2 = corr2d_multi_in_out(X, K)
assert float(torch.abs(Y1 - Y2).sum()) < 1e-6

### 6.4.5 练习

1. 假设我们有两个卷积核，大小分别为 $k_1$ 和 $k_2$（中间没有非线性激活函数）。
    1. 证明运算可以用单次卷积来表示。
        - 证明：两个卷积核的卷积运算可以合并为一个等效的卷积核。设卷积核 $k_1$ 和 $k_2$ 分别作用于输入 $X$，则有：
          $$[
          Y = (X * k_1) * k_2
          ]$$
          根据卷积的结合律，可以将其表示为：
          $$[
          Y = X * (k_1 * k_2)
          ]$$
          因此，两个卷积核的运算可以用一个等效的卷积核 $k = k_1 * k_2$ 来表示。
    2. 这个等效的单个卷积核的维数是多少呢？
        - 等效卷积核的维数为 $k_1 + k_2 - 1$。
    3. 反之亦然吗？
        - 是的，反之亦然。即一个卷积核可以分解为两个卷积核的卷积。

2. 假设输入为 $c_i \times h \times w$，卷积核大小为 $c_o \times c_i \times k_h \times k_w$，填充为 $(p_h, p_w)$，步幅为 $(s_h, s_w)$。
    1. 前向传播的计算成本（乘法和加法）是多少？
        - 计算成本为 $c_o \times c_i \times k_h \times k_w \times \frac{(h + 2p_h - k_h)}{s_h + 1} \times \frac{(w + 2p_w - k_w)}{s_w + 1}$。
    2. 内存占用是多少？
        - 内存占用为 $c_o \times \frac{(h + 2p_h - k_h)}{s_h + 1} \times \frac{(w + 2p_w - k_w)}{s_w + 1}$。
    3. 反向传播的内存占用是多少？
        - 反向传播的内存占用与前向传播相同。
    4. 反向传播的计算成本是多少？
        - 反向传播的计算成本与前向传播相同。

3. 如果我们将输入通道 $c_i$ 和输出通道 $c_o$ 的数量加倍，计算量会增加多少？如果我们把填充数量翻一番会怎么样？
    - 如果输入通道和输出通道的数量加倍，计算量会增加 4 倍。
    - 如果填充数量翻一番，计算量会增加，但具体增加量取决于填充的具体数值。

4. 如果卷积核的高度和宽度是 $k_h = k_w = 1$，前向传播的计算复杂度是多少？
    - 计算复杂度为 $c_o \times c_i \times h \times w$。

5. 本节最后一个示例中的变量 Y1 和 Y2 是否完全相同？为什么？
    - 是的，Y1 和 Y2 完全相同。因为 1x1 卷积等价于全连接层的矩阵乘法，结果是相同的。

6. 当卷积窗口不是 1x1 时，如何使用矩阵乘法实现卷积？
    - 当卷积窗口不是 1x1 时，可以使用 `im2col` 方法将卷积操作转换为矩阵乘法。具体步骤如下：
        1. 将输入张量展开为一个矩阵，每一列对应一个卷积窗口。
        2. 将卷积核展开为一个矩阵。
        3. 进行矩阵乘法，得到卷积结果。
